In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.rcParams['figure.figsize']=(7,4)  # figure 사이즈 변경
sns.set_palette('twilight')
pal_1=sns.color_palette('coolwarm', 10)
pal_2=sns.color_palette('deep',10)

from matplotlib import font_manager, rc
font = 'C:/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname=font).get_name()
rc('font', family=font_name)

#### 신경망: 분류   - 은닉층 없음(퍼셉트론)

In [ ]:
### 클라스 만들어서 전체 해보기
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
# 스케일링
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
class SingleLayer:
    def __init__(self):
        self.w = None
        self.b = None
        self.losses = []
    def forpass(self, x):
        z = np.dot(x, self.w) + self.b
        return z

    def backprop(self, x, err):
        m = len(x)
        w_grad = np.dot(x.T, err) / m
        b_grad = np.sum(err) / m
        return w_grad, b_grad
    def activation(self, z):
        z = np.clip(z, -100, None)
        a = 1 / (1 + np.exp(-z))
        return a
    def predict(self, x):
        z = self.forpass(x)
        return z > 0
    def score(self, x, y):
        return np.mean(self.predict(x) == y.reshape(-1, 1))
    def fit(self, x, y, epochs = 100, random_state = None):
        y = y.reshape(-1,1) # 열 벡터로 변환
        m = len(x)
        self.w = np.ones((x.shape[1], 1)) # 가중치 초기화
        self.b = 0 # 절편 초기화
        for i in range(epochs):
            z = self.forpass(x)
            a = self.activation(z)
            err = -(y - a)
            w_grad, b_grad = self.backprop(x, err)
            self.w-= w_grad
            self.b-= b_grad
            a = np.clip(a, 1e-10, 1-1e-10)
            loss = np.mean(-(y*np.log(a) + (1-y)*np.log(1-a))) 
            self.losses.append(loss)


In [ ]:
single_layer = SingleLayer()
single_layer.fit(X_train_scaled, y_train)
single_layer.score(X_test_scaled, y_test)

In [ ]:
plt.plot(single_layer.losses)
plt.ylim(0,0.1)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()
print(single_layer.losses[-1])

In [ ]:
#### 분류 손실함수ㅡ 활성화 함수: 이진분류(시그모이드), 다중분류(soft max)
# 손실함수의 결과값 저장 기능: 손실함수(loss) 계산하기
# 손실함수(분류) - logistic// cross Entropy

class SingleLayer:  # 입력계층 1개, 은닉층 없음
    def __init__(self):
        self.w = None    # 입력된 값 없으므로
        self.b = None
        # 손실 함수 저장하기 위한 리스트
        self.losses = []
    def forpass(self, x):
        z = np.sum(x * self.w) + self.b
        return z
    def backprop(self, x, err):
        w_grad = x * err
        b_grad = 1 * err
        return w_grad, b_grad
    def activation(self, z):   ### 여길 바꿈
        z = np.clip(z, -100, None)
        a = 1 / (1 + np.exp(-z))   # 시그모이드 활성함수/이진분류
        return a
    def fit(self, x, y, epochs = 100):
        self.w = np.ones(x.shape[1])
        self.b = 0 
        for epoch in range(epochs):
            # 손실 초기화
            loss = 0
            # x의 index 랜덤하게 반환/ x,y가 각각 섞이지않도록 인덱스를 만들어 행렬로 만들어 사용
            indexes = np.random.permutation(np.arange(len(x)))
            for i in indexes:
                z = self.forpass(x[i])
                a = self.activation(z)
                err = -(y[i] - a)
                w_grad, b_grad = self.backprop(x[i], err)
                self.w -= w_grad
                self.b -= b_grad
                 # 안전한 로그 계산을 위한 범위 축소  
                a = np.clip(a, 1e-10, 1-1e-10)
                # 이걸 회귀에서 바꿔줌: 손실 계산 : MSE 
                loss += -(y[i] * np.log(a) + (1 - y[i]) * np.log(1 - a))
            # 에포크마다 평균 손실을 저장
            self.losses.append(loss / len(y))  # loss값의 평균값(로스합계/y갯수)을 저장
            print(f'###### 회귀 EPOCHS : {epoch + i}  #########')
            print(f'평균 loss : {loss}')
    def predict(self, x):
        z = [self.forpass(x_i) for x_i in x]
        return z
    # 정확도 계산 함수 생성
    def score(self, x, y):
        return np.mean(self.predict(x) == y)

#### 신경망 : 회귀  - 은닉층 없음(퍼셉트론)

In [ ]:
### 회귀에서 손실함수 저장하기  : 분류와 다른점 -활성함수(출력층)/ 손실함수 관련해서 바꿀것
# 회귀: 활성함수 없음 또는 y=x
# 손실함수의 결과값 저장 기능: 손실함수(loss) 계산하기 // MSE

class SingleLayerRegression: # 입력계층 1개, 은닉층 없음
    def __init__(self):
        self.w = None   # 입력된 값 없으므로
        self.b = None
        # 손실 함수 저장하기 위한 리스트
        self.losses = []
        
    def forpass(self, x):
        z = np.sum(x * self.w) + self.b
        return z
    
    def backprop(self, x, err):
        w_grad = x * err
        b_grad = 1 * err
        return w_grad, b_grad
    
    def activation(self, z):    #### 이걸 바꿔줘야
        a = z
        return a
    
    def fit(self, x, y, epochs = 100):
        self.w = np.ones(x.shape[1])
        self.b = 0 
        for epoch in range(epochs):
            # 손실 초기화
            loss = 0
            # x의 index 랜덤하게 반환
            indexes = np.random.permutation(np.arange(len(x)))
            for i in indexes:
                z = self.forpass(x[i])
                a = self.activation(z)
                err = -(y[i] - a)  # (y - y_hat)
                w_grad, b_grad = self.backprop(x[i], err)
                self.w -= w_grad
                self.b -= b_grad
                
                # 손실 계산 (SE)  (y - y_hat)^2
                loss += err ** 2

            # 에포크마다 평균 손실을 저장
            loss = loss / len(y) # MSE
            self.losses.append(loss)
            print(f'##### EPOCHS : {epoch + 1} ######')
            print(f'loss : {loss}')
            
    def predict(self, x):
        z = [self.forpass(x_i) for x_i in x]
        return z


#### 다층신경망 : 분류  - 은닉층 1개

In [ ]:
## 다층 신경망
class DualLayer:     # 은닉층 1개
    def __init__(self, units = 8):  # 클래스 만들때 모든 은닉층, 가중치 등 정해 틀 만들고 시작
        self.units = units # 은닉층의 뉴런 개수
        self.w1 = None    # 입력 > 은닉 가중치
        self.b1 = None    # 입력 > 은닉 절편
        self.w2 = None    # 은닉 > 출력 가중치
        self.b2 = None    # 은닉 > 출력 절편
        self.a1 = None    # 은닉층의 활성화 출력
        self.losses = []
    def forpass(self, x):
        z1 = np.dot(x, self.w1) + self.b1
        self.a1 = self.activation(z1)            # 상속 받아 활성함수는 시그모이드
        z2 = np.dot(self.a1, self.w2) + self.b2  # np.dot 행렬
        return z2
    def backprop(self, x, err):
        m = len(x)
        # 은닉층 > 출력층 가중치, 절편 업데이트
        w2_grad = np.dot(self.a1.T, err) / m    # 평균
        b2_grad = np.sum(err) / m               # 평균
        # 은닉층 오차
        err_to_hidden = np.dot(err, self.w2.T) * self.a1 * (1 - self.a1)
        # 입력층 > 은닉층 가중치, 절편 업데이트        
        w1_grad = np.dot(x.T, err_to_hidden) / m
        b1_grad = np.sum(err_to_hidden, axis=0) / m
        return w1_grad, b1_grad, w2_grad, b2_grad
    def activation(self, z):
        z = np.clip(z, -100, None)
        a = 1 / (1 + np.exp(-z))
        return a
    def predict(self, x):     # y 결과 값
        z = self.forpass(x)
        return z > 0
    def score(self, x, y):
        return np.mean(self.predict(x) == y.reshape(-1, 1))  
  
    def init_weights(self, n_features):        # 가중치 초기화  
        self.w1 = np.ones((n_features, self.units))  # 2차원으로 만들어야 행렬곱 가능
        self.b1 = np.zeros(self.units)
        self.w2 = np.ones((self.units, 1))
        self.b2 = 0
    def training(self, x, y, m): ## 마지막 활성함수므로 분류할지 회귀할지 정하고 코드수정할것          
        z = self.forpass(x)
        a = self.activation(z)
        err = -(y - a)
        w1_grad, b1_grad, w2_grad, b2_grad = self.backprop(x, err)
        self.w1 -= w1_grad 
        self.b1 -= b1_grad
        self.w2 -= w2_grad 
        self.b2 -= b2_grad
        return a
    def fit(self, x, y, epochs = 100):  # 
        y = y.reshape(-1,1)
        m = len(x)
        self.init_weights(x.shape[1])
        for i in range(epochs):
            a = self.training(x, y, m)
            a = np.clip(a, 1e-10, 1-1e-10)
            loss = np.sum(-(y * np.log(a) + (1 - y) * np.log(1 - a)))
            self.losses.append(loss / m)

In [ ]:
dual_layer = DualLayer()
dual_layer.fit(X_train_scaled, y_train, epochs=1000)
print(dual_layer.score(X_test_scaled, y_test))

plt.plot(dual_layer.losses)
plt.ylim(0,0.6)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()
print(dual_layer.losses[-1])

#### 다중 분류 클래스 

In [ ]:
## 다중 분류 클래스 만들기  : 
# 다 똑같은데, 다른 점 마지막 활성함수(소프트맥스), 손실함수(크로스 엔트로피)

class MultiClassNetwork:
    def __init__(self, units = 10): # 전과 동일
        self.units = units 
        self.w1 = None 
        self.b1 = None 
        self.w2 = None 
        self.b2 = None 
        self.a1 = None 
        self.losses = []
    def forpass(self, x): 
        z1 = np.dot(x, self.w1) + self.b1
        self.a1 = self.sigmoid(z1)             # 활성 함수 이름 변경
        z2 = np.dot(self.a1, self.w2) + self.b2 
        return z2
    def backprop(self, x, err): # 전과 동일
        m = len(x)
        w2_grad = np.dot(self.a1.T, err) / m
        b2_grad = np.sum(err) / m
        err_to_hidden = np.dot(err, self.w2.T) * self.a1 * (1 - self.a1)
        w1_grad = np.dot(x.T, err_to_hidden) / m
        b1_grad = np.sum(err_to_hidden, axis=0) / m
        return w1_grad, b1_grad, w2_grad, b2_grad
    def sigmoid(self, z): # 시그모이드 함수
        z = np.clip(z, -100, None)
        a = 1 / (1 + np.exp(-z))
        return a
    def softmax(self, z):     #소프트맥스 함수
        z = np.clip(z, -100, None)
        exp_z = np.exp(z)
        return exp_z / np.sum(exp_z, axis = 1).reshape(-1, 1)
    def init_weights(self, n_features, n_classes): # 클래스 개수 받음
        np.random.seed(0)
        self.w1 = np.random.normal(0, 1, (n_features, self.units))
        self.b1 = np.zeros(self.units)
        self.w2 = np.random.normal(0, 1, (self.units, n_classes)) # 클래스 개수 포함
        self.b2 = np.zeros(n_classes) # 클래스 개수 포함
    def training(self, x, y, m):
        z = self.forpass(x)
        a = self.softmax(z) # 출력 계층 활성 함수 변경
        err = -(y - a)      # 에러값 3개 나옴
        w1_grad, b1_grad, w2_grad, b2_grad = self.backprop(x, err)
        self.w1 -= w1_grad
        self.b1 -= b1_grad
        self.w2 -= w2_grad
        self.b2 -= b2_grad
        return a
    def fit(self, x, y, epochs = 100):
        m = len(x)
        self.init_weights(x.shape[1], y.shape[1]) # 가중치 초기화시 클래스 개수 포함
        for I in range(epochs): 
            print('.', end='') # epochs 1번마다 . 찍음
            a = self.training(x, y, m)
            a = np.clip(a, 1e-10, 1-1e-10)
            loss = np.sum(-y * np.log(a)) # 크로스 엔트로피 손실 함수
            self.losses.append(loss / m)
    def predict(self, x):
        z = self.forpass(x)
        return np.argmax(z, axis = 1) # 예측한 결과에서 가장 큰 확률 인덱스
    def score(self, x, y):
        # 정답 인덱스 확인
        return np.mean(self.predict(x) == np.argmax(y, axis = 1)) 
                            

In [ ]:
# 이미지 데이터로 실습 (내일 합시당)

import tensorflow as tf
import matplotlib.pyplot as plt
# 데이터 불러오기
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
# 데이터 확인하기
plt.imshow(X_train[0], cmap='gray')
plt.colorbar()
plt.show()
print(X_train[0][:5])